## Interview Query Question #21 | First Touch Attribution

Good morning. Here's your sql question for today.. Try out solving the problem first before clicking on the link to the solution.

This problem will be free for a week, then it'll go back to being available only in our premium course.

This question was asked by: Google  
`attribution` table  

column	type  
id	int  
created_at	datetime  
session_id	int  
channel	varchar  
conversion	boolean  

`user_sessions` table    

column	user_id  
session_id	int  
user_id	int  
 

The schema above is for a retail online shopping company consisting of two tables, attribution and user_sessions. 

The attribution table logs a session visit for each row.
If conversion is true, then the user converted to buying on that session.
The channel column represents which advertising platform the user was attributed to for that specific session.
Lastly the `user_sessions` table maps many to one session visits back to one user.
First touch attribution is defined as the channel to which the converted user was associated with when they first discovered the website.

Calculate the first touch attribution for each user_id that converted.

 

Example output:  

user_id	channel  
123	facebook  
145	google  
153	facebook  
172	organic  
173	email  
Need a hint?
Imagine the full path of a user converting to a purchase after multiple visits.

1st Session: User sees Facebook ad -> Clicks to order -> Leaves
2nd Session: User sees Google ad -> Leaves
3rd Session: User types in website -> Clicks to order -> Purchases
How do we figure out the beginning path of the Facebook ad and connect it to the end purchasing user?

In [ ]:
# idea: rownumber window function over user partition

with win_sessions as (
select user_id, session_id, created_at, channel, conversion, rownumber() over partition by user_id order by created_at asc as row
from attribution a join user_sessions s on a.session_id = s.session_id
)
,
converted_users as (
select user_id
from attribution a join user_sessions s on a.session_id = s.session_id
where conversion = True
)

select user_id, channel
from win_sessions
where user_id in (select user_id from converted_users) and row = 1
